## 9.6 Deutsch algorithm (part 2)
A demonstration of the [Deutsch algorithm](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm).

In [ ]:
using ImageShow
using StrangelyDisplayed
using StrangelyQuantum

A function to create one of four oracle gates (identical to the function provided [earlier](ch09-04-applyoracle.ipynb):

In [ ]:
function createOracle(f)
    matrix = zeros(ComplexF64, 4, 4)

    if f == 1
        matrix[1, 1] = 1
        matrix[2, 2] = 1
        matrix[3, 3] = 1
        matrix[4, 4] = 1
        return Oracle(matrix)
    elseif f == 2
        matrix[1, 1] = 1
        matrix[2, 4] = 1
        matrix[3, 2] = 1
        matrix[4, 3] = 1
        return Oracle(matrix)
    elseif f == 3
        matrix[1, 3] = 1
        matrix[2, 2] = 1
        matrix[3, 1] = 1
        matrix[4, 4] = 1
        return Oracle(matrix)
    elseif f == 4
        matrix[1, 3] = 1
        matrix[2, 4] = 1
        matrix[3, 1] = 1
        matrix[4, 2] = 1
        return Oracle(matrix)
    else
        throw("Wrong index in oracle")
    end
end

A new driver for the oracle function that uses Deutsch's algorithm on the four oracle gates:

In [ ]:
function deutsch()
    simulator = SimpleQuantumExecutionEnvironment()
    program = nothing
    for choice = 1:4
        # Create a program with two qubits
        program = Program(2)
        step0 = Step()
        # Apply a Pauli-X gate to the second qubit.
        addGate(step0, X(2))

        step1 = Step()
        # Apply Hadamard gates to both qubits.
        addGate(step1, Hadamard(1))
        addGate(step1, Hadamard(2))

        step2 = Step()
        # Choose an oracle (from a predefined
        # list) and apply it
        oracle = createOracle(choice)
        addGate(step2, oracle)

        step3 = Step()
        # Apply another Hadamard gate to the first qubit
        addGate(step3, Hadamard(1))

        addStep(program, step0)
        addStep(program, step1)
        addStep(program, step2)
        addStep(program, step3)
        # Execute the quantum program
        result = runProgram(simulator, program)
        qubits = getQubits(result)
        # Measure the first qubit is. Based on its
        # value, we know whether the oracle corresponded
        # with a constant or balanced function.
        println("f = ", choice, ", val = ", measure(qubits[1]))
    end
    # return the final program
    return program
end

Run the algoritm to get the results.

In [ ]:
program = deutsch()

Functions 1 and 4 are constant functions, and this is what the program confirms. The constructed program for function 4 and its sampling histogram are show below.

In [ ]:
drawProgram(program)

In [ ]:
drawTrialHistogram(program, 1000)